In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)

data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [2]:
import itertools
from tqdm import tqdm
from datetime import datetime, timedelta
pd.set_option('mode.chained_assignment',  None)
class Processing:
    def __init__(self, x, test): # 이후 테스트 데이터도 넣는 버전 만들어야함
        self.x = x
        self.test = test
    
    def fill_airline_and_id(self, train, test):
        print('Fill Airline and ID')
        dum = train.dropna()
        dum = dum.groupby('Carrier_ID(DOT)')['Airline'].value_counts().to_frame().rename(columns={'Airline': 'count'}).reset_index()
        
        for i in tqdm(range(dum.shape[0])):
            id = dum['Carrier_ID(DOT)'].iloc[i]
            airline = dum['Airline'].iloc[i]
            
            train.loc[train['Carrier_ID(DOT)'] == id, 'Airline'] = airline
            test.loc[test['Carrier_ID(DOT)'] == id, 'Airline'] = airline
            
            train.loc[train['Airline'] == airline, 'Carrier_ID(DOT)'] = id
            test.loc[test['Airline'] == airline, 'Carrier_ID(DOT)'] = id 
            
        print('Done')        
            
        return train, test
    
    def fill_state(self, train, test):
        dum = train.dropna()
        dum1 = dum.groupby('Origin_State')['Origin_Airport_ID'].value_counts().to_frame().rename(columns={'Origin_Airport_ID': 'count'}).reset_index()
        dum2 = dum.groupby('Destination_State')['Destination_Airport_ID'].value_counts().to_frame().rename(columns={'Destination_Airport_ID': 'count'}).reset_index()
        
        print('Fill State')
        print('Origin State')
        for i in tqdm(range(dum1.shape[0])):
            state = dum1['Origin_State'].iloc[i]
            id = dum1['Origin_Airport_ID'].iloc[i]
            
            train.loc[train['Origin_Airport_ID'] == id, 'Origin_State'] = state
            test.loc[test['Origin_Airport_ID'] == id, 'Origin_State'] = state
            
        print('Destination State')
        for i in tqdm(range(dum2.shape[0])):
            state = dum2['Destination_State'].iloc[i]
            id = dum2['Destination_Airport_ID'].iloc[i]
            
            train.loc[train['Destination_Airport_ID'] == id, 'Destination_State'] = state
            test.loc[test['Destination_Airport_ID'] == id, 'Destination_State'] = state
        
        print('Done')        
        
        return train, test
   
    def run(self):
        train, test = self.fill_airline_and_id(self.x, self.test)
        train, test = self.fill_state(train, test)

        return train, test

    
    


    
                
                
            
        

In [3]:
p = Processing(data, test)
train, test = p.run()

Fill Airline and ID


100%|██████████| 28/28 [00:02<00:00, 11.19it/s]


Done
Fill State
Origin State


100%|██████████| 371/371 [00:02<00:00, 135.13it/s]


Destination State


100%|██████████| 373/373 [00:02<00:00, 133.46it/s]

Done


In [4]:
train.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     6
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                1
Distance                         0
Airline                      11883
Carrier_Code(IATA)          108990
Carrier_ID(DOT)              11883
Tail_Number                      0
Delay                       744999
dtype: int64

In [5]:
test.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                    39
Destination_Airport              0
Destination_Airport_ID           0
Destination_State               22
Distance                         0
Airline                      11543
Carrier_Code(IATA)          108993
Carrier_ID(DOT)              11543
Tail_Number                      0
dtype: int64